In [3]:
import folium
import openrouteservice as ors


# Your OpenRouteService API key
api_key = api_key
client = ors.Client(key=api_key)

# Define the addresses
addresses = [
    "1035 Lawrence Dr, Burlington",
    "6421 University Ave, Cedar Falls",
    "3580 8th St SW, Altoona",
    "1310 S. Duff Ave, Ames",
    "2310 SE Delaware Ave, Ankeny",
    "4701 1st Ave SE, Cedar Rapids",
    "5300 Edgewood Rd NE, Cedar Rapids",
    "2315 Edgewood Rd SW, Cedar Rapids",
    "5350 Kirkwood Blvd, Cedar Rapids",
    "901 25th Ave, Coralville",
    "902 W Kimberly Rd, Davenport",
    "4888 Utica Ridge Rd, Davenport", 
    "2708 Ingersoll Ave, Des Moines",
    "4840 Ashbury Rd, Dubuque",
    "3024 5th Ave S, Fort Dodge",
    "2250 East 1st St, Grimes",
    "32 S Clinton St, Iowa City",
    "965 S Riverside Dr, Iowa City"
    "8759 Northpark Dr, Johnston", 
    "6095 Carlson Way, Marion",
    "2261 8th St, SW Mason City",
    "650 Community Dr, North Liberty",
    "3265 100th St, Urbandale",
    "2845 Crossroads Blvd, Waterloo",
    "757 E. Hickman Rd, Waukee",
    "165 S Jordan Creek Pkwy, West Des Moines"
]

coordinates = []

# Geocoding using openrouteservice
for address in addresses:
    response = client.pelias_search(text=address, size=1)
    if response and 'features' in response and len(response['features']) > 0:
        coords = response['features'][0]['geometry']['coordinates']
        coordinates.append(coords)
    else:
        print(f"Could not find coordinates for {address}")

# Map initialization
m = folium.Map(location=list(reversed(coordinates[0])), tiles="cartodbpositron", zoom_start=8)

# Add markers for addresses
for coord in coordinates:
    folium.Marker(location=list(reversed(coord))).add_to(m)

# Vehicle Start Marker
vehicle_start = [-93.603729, 41.591099]
folium.Marker(location=list(reversed(vehicle_start)), icon=folium.Icon(color="red")).add_to(m)

# Create jobs for optimization
jobs = [
    ors.optimization.Job(id=index, location=coord, service=1)
    for index, coord in enumerate(coordinates)
]

# Vehicle configuration
vehicle = [
    ors.optimization.Vehicle(
        id=0,
        profile='driving-car',
        start=vehicle_start,
        end=vehicle_start
    )
]

# Perform optimization
optimized = client.optimization(
    jobs=jobs,
    vehicles=vehicle,
    geometry=True
)

# Plot optimized routes on the map
line_colors = ['orange']
for route in optimized['routes']:
    coordinates = ors.convert.decode_polyline(route['geometry'])['coordinates']
    folium.PolyLine(locations=[list(reversed(coord)) for coord in coordinates], color=line_colors[route['vehicle']]).add_to(m)
    
    
# Display the map

m


KeyError: 'summary'